In [2]:
import librosa
import numpy as np
import wave
import json
from vosk import Model, KaldiRecognizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
from transformers import AutoTokenizer, AutoModel
import torch
import os

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

y, sr = librosa.load("5 Britten Close.m4a")
# Use librosa's piptrack for pitch tracking
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

# Extract the strongest pitch per frame
pitch_values = []
for i in range(pitches.shape[1]):
    index = magnitudes[:, i].argmax()
    pitch = pitches[index, i]
    if pitch > 0:  # Remove zero or negative values
        pitch_values.append(pitch)

pitch_values = np.array(pitch_values)
print("Mean pitch (Hz):", np.mean(pitch_values))


/var/folders/5p/j0ltycyj5b19sy6yyqzr7q500000gn/T/ipykernel_99627/222396416.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("5 Britten Close.m4a")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Mean pitch (Hz): 320.13388


In [4]:
# Estimate jitter (approximate)
if len(pitch_values) > 1:
    diffs = np.abs(np.diff(pitch_values))
    jitter_local = np.mean(diffs / pitch_values[:-1])
    print("Approximate Jitter (local):", jitter_local)
else:
    print("Insufficient pitch data for jitter estimation.")


Approximate Jitter (local): 0.34649768


In [5]:


# Compute short-time RMS energy
rms = librosa.feature.rms(y=y)[0]

# Estimate shimmer as the mean absolute percentage difference between frames
if len(rms) > 1:
    shimmer_values = np.abs(np.diff(rms)) / rms[:-1]
    shimmer_local = np.mean(shimmer_values)
    print("Approximate Shimmer (local):", shimmer_local)
else:
    print("Insufficient RMS data for shimmer estimation.")


Approximate Shimmer (local): 0.20525113


In [6]:
# 4. Speech rate and pauses
intervals = librosa.effects.split(y, top_db=20)
speech_durations = sum([(end - start) / sr for start, end in intervals])
total_duration = len(y) / sr
speech_ratio = speech_durations / total_duration
print("Speech Duration:", speech_durations)
print("Speech Rate (ratio):", speech_ratio)

Speech Duration: 8.661043083900225
Speech Rate (ratio): 0.7463819376321476


In [13]:
from pydub import AudioSegment

# Convert M4A to WAV
input_audio_path = "5 Britten Close.m4a"
AUDIO_PATH = "converted.wav"

if not os.path.exists(AUDIO_PATH):
    audio = AudioSegment.from_file(input_audio_path, format="m4a")
    audio.export(AUDIO_PATH, format="wav")

In [19]:
from pydub import AudioSegment

input_path = "5 Britten Close.m4a"
output_path = "converted.wav"

sound = AudioSegment.from_file(input_path)
sound = sound.set_channels(1)
sound = sound.set_frame_rate(16000)
sound.export(output_path, format="wav")

<_io.BufferedRandom name='converted.wav'>

In [ ]:

wf = wave.open(AUDIO_PATH, "rb")
vosk_model_path = "model"  # VOSK model directory (download from https://alphacephei.com/vosk/models)
if not os.path.exists(vosk_model_path):
    print("Please download and extract VOSK model to 'model/' folder.")
model = Model(vosk_model_path)
rec = KaldiRecognizer(model, wf.getframerate())

results = []

while True:
    data = wf.readframes(8000)
    if len(data) == 0:
        break

    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        print("✅ Full:", result)
        results.append(result.get("text", ""))
    else:
        partial = json.loads(rec.PartialResult())
        # print("🟡 Partial:", partial)

# Final catch
final_result = json.loads(rec.FinalResult())
results.append(final_result.get("text", ""))

transcript = " ".join(results)
print("\n📝 Final Transcript:\n", transcript)


import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('punkt')
words = word_tokenize(transcript)
sentences = sent_tokenize(transcript)
avg_sentence_length = np.mean([len(word_tokenize(s)) for s in sentences])
vocab_richness = len(set(words)) / len(words)
word_freq = FreqDist(words).most_common(10)
print("Average Sentence Length:", avg_sentence_length)
print("Vocabulary Richness:", vocab_richness)
print("Top Words:", word_freq)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from model/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from model/graph/HCLr.fst model/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo model/graph/phones/word_boundary.int



📝 Final Transcript:
 the hello my name is my name is or where clinton plan plan and this is our dog this is for testing testing testing
Average Sentence Length: 24.0
Vocabulary Richness: 0.625
Top Words: [('is', 4), ('testing', 3), ('my', 2), ('name', 2), ('plan', 2), ('this', 2), ('the', 1), ('hello', 1), ('or', 1), ('where', 1)]
